In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, regexp_replace, when

spark = SparkSession.builder.appName("CarDataProcessing").getOrCreate()

file_path = r"C:/Users/acdsa/Desktop/BDA/lab2/Lab2/lab3/car_price_dataset.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

print("Original Dataset:")
df.show(5)

Original Dataset:
+----------+------+----+-----------+---------+--------------+-------+-----+-----------+-----+
|     Brand| Model|Year|Engine_Size|Fuel_Type|  Transmission|Mileage|Doors|Owner_Count|Price|
+----------+------+----+-----------+---------+--------------+-------+-----+-----------+-----+
|       Kia|   Rio|2020|        4.2|   Diesel|        Manual| 289944|    3|          5| 8501|
| Chevrolet|Malibu|2012|        2.0|   Hybrid|     Automatic|   5356|    2|          3|12092|
|  Mercedes|   GLA|2020|        4.2|   Diesel|     Automatic| 231440|    4|          2|11171|
|      Audi|    Q5|2023|        2.0| Electric|        Manual| 160971|    2|          1|11780|
|Volkswagen|  Golf|2003|        2.6|   Hybrid|Semi-Automatic| 286618|    3|          3| 2867|
+----------+------+----+-----------+---------+--------------+-------+-----+-----------+-----+
only showing top 5 rows



In [3]:
df_cleaned = df.dropDuplicates()

df_cleaned = df_cleaned.withColumn("price", regexp_replace(col("price"), "[^0-9.]", ""))
df_cleaned = df_cleaned.withColumn("year", regexp_replace(col("year"), "[^0-9]", ""))

df_cleaned = df_cleaned.withColumn("price", when(col("price") == "", "0").otherwise(col("price")))
df_cleaned = df_cleaned.withColumn("year", when(col("year") == "", "0").otherwise(col("year")))

# Corrected the typo to use 'fillna' instead of 'fillNa'
df_cleaned = df_cleaned.fillna({"price": "0", "brand": "Unknown", "year": "0"})

df_cleaned = df_cleaned.withColumn("price", col("price").cast("float"))
df_cleaned = df_cleaned.withColumn("year", col("year").cast("int"))

In [4]:
df_filtered = df_cleaned.filter(col("price") > 5000)

print("Cleaned and Filtered Dataset:")
df_filtered.show(5)

Cleaned and Filtered Dataset:
+----------+------+----+-----------+---------+------------+-------+-----+-----------+-------+
|     Brand| Model|year|Engine_Size|Fuel_Type|Transmission|Mileage|Doors|Owner_Count|  price|
+----------+------+----+-----------+---------+------------+-------+-----+-----------+-------+
| Chevrolet|Impala|2013|        4.8|   Petrol|      Manual| 206381|    4|          4| 8672.0|
|    Toyota| Camry|2023|        1.5|   Diesel|      Manual| 117049|    2|          5|10159.0|
|      Audi|    A3|2005|        2.3| Electric|      Manual| 112828|    3|          2| 7643.0|
|Volkswagen|Passat|2007|        3.0| Electric|      Manual| 272439|    3|          3| 5751.0|
|Volkswagen|  Golf|2010|        2.9| Electric|      Manual| 249910|    5|          3| 7001.0|
+----------+------+----+-----------+---------+------------+-------+-----+-----------+-------+
only showing top 5 rows



In [5]:
df_grouped = df_filtered.groupBy("brand").agg(avg("price").alias("Average_Price"))

print("Average Price by Brand:")
df_grouped.show()

Average Price by Brand:
+----------+-----------------+
|     brand|    Average_Price|
+----------+-----------------+
|Volkswagen|9610.589800443458|
|       Kia|9483.450285714285|
| Chevrolet|9649.525139664804|
|   Hyundai|9502.603686635945|
|     Honda|9349.710585585586|
|      Audi|9682.598233995584|
|  Mercedes|9653.273053892215|
|       BMW|9459.895402298851|
|    Toyota|  9585.8138424821|
|      Ford|9528.752155172413|
+----------+-----------------+



In [7]:
output_path = r"C:/Users/acdsa/Desktop/BDA/lab2/Lab2/lab3/car_price_output.csv"

# Convert to Pandas and save as a single CSV
df_grouped.toPandas().to_csv(output_path, index=False)

print(f"Saved file path: {output_path}")


Saved file path: C:/Users/acdsa/Desktop/BDA/lab2/Lab2/lab3/car_price_output.csv
